In [56]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
import sys 
sys.path.append('..')

In [58]:
import time
import pickle 
import numpy as np 
import pandas as pd
import networkx as nx 

from juliacall import Main as jl  
from hdna import *

In [59]:
# Import experimental data inside a dataframe

expdata = pd.read_csv('../data/herteldata.csv', names=['seq', 'expvalue'])

In [60]:
# Clean the dataframe 

expdata = expdata.drop(0)
expdata['expvalue'] = ['{:e}'.format(float(e)) for e in expdata['expvalue']]

In [61]:
# Actual computation 
EXPNAME = 'datawritingtrials'
rates = []

model = Model('dna', '3D')

for i, (seq, exp) in enumerate(zip(expdata['seq'], expdata['expvalue'])):
    seq = str(seq.strip())      # make sure they are alright
    exp = float(exp.strip())    #
    print(f'Creating network from string {seq}...')
    A = Strand(model, seq)
    B = A.complementary()
    kinet = Kinetwork(model, A, B, 1)
    geo = Geometry(120, 360)
    K = Kinetics(model, kinet, geo)
    opts = Options(method='direct', runtime=3e-6, Nsim=200, results_dir=f'../results/{EXPNAME}')
    print('embedding strand into biosimulator network model...')
    simulatore = Simulator(model, kinet, K, options=opts)
    print('start running simulations...')
    results = simulatore.ensemble()
    mfpt = simulatore.mfpts(results)
    rates.append(1/mfpt)
    print(f"experimental rate: {'{:e}'.format(exp)}")
    print(f"computed rate:     {'{:e}'.format(1/mfpt)}")
    break 

Creating network from string GTTCGGTCTA...
embedding strand into biosimulator network model...
start running simulations...


100%|██████████| 200/200 [00:00<00:00, 1585.54it/s]


34 simulations didn't produce a duplex.
That's 17.0% of simulations
experimental rate: 1.145950e+06
computed rate:     8.782549e+05


In [64]:
# DG = simulatore.digraph
# for n in DG.nodes.data():
#     n[1]['object'] = str(type(n[1]['object']))
# nx.write_gexf(DG,'digraphtrial.gexf')

# # str(type(list(DG.nodes.data())[0][1]['object']))

In [65]:
# states = [node[1]['state'] for node in list(DG.nodes.data())]
# visnet = Network(directed=True,select_menu=True,filter_menu=True, heading = 'stringseq')
# visnet.add_nodes(list(DG.nodes()), state=states)
# for edge in list(DG.edges.data()):
#     visnet.add_edge(edge[0], edge[1], **edge[2])
# visnet.show_buttons()
# visnet.toggle_physics(True)
# visnet.show('name.html')